### Initial imports and configurations

In [ ]:
import shutil
import requests
import urllib3

s = requests.session()
s.verify=False
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

keycloak_host="https://myhost" 
deployer_host="https://deployer.local.lan"
username="kilar0276"
password="Test1234"

### Get access token
Access token is valid for 5 minutes

In [ ]:
r = s.post(f"{keycloak_host}/keycloak/realms/ctf/protocol/openid-connect/token/", data={"client_id":"deployer", "username": username, "password": password, "grant_type": "password", "scope": "openid"}, timeout=20)
print("login:", r.status_code, r.content)
r.raise_for_status()
s.headers = {"Authorization": "Bearer " + r.json().get("access_token")}

### Post challenge to deployer service

In [ ]:
shutil.make_archive("challenge", "zip", "../challenge/src/")
shutil.make_archive("solution", "zip", "../challenge/solution/")
r = s.post(deployer_host + "/challenges", files=[
    ("upload[]", open("challenge.zip", "rb")),
    ("upload[]", open("../challenge/challenge.yml", "rb")),
     ("upload[]", open("solution.zip", "rb"))], timeout=20)
print("add challenge:", r.status_code, r.content)
r.raise_for_status()
challenge_id = r.json().get("challengeid")

### Submit challenge to CTFd

In [ ]:
r = s.post(deployer_host + "/challenges/" + challenge_id + "/publish", timeout=20)
print("publish:", r.status_code, r.content)

### Start the challenge using the API

In [ ]:
r = s.post(deployer_host + "/challenges/" + challenge_id + "/start", timeout=20)
print("start challenge:", r.status_code, r.content)
r.raise_for_status()

### Start test using the API

In [ ]:
r = s.post(deployer_host + f"/solutions/{challenge_id}/start", timeout=20)
print("start test:", r.status_code, r.content)
r.raise_for_status()

### Verify the submitted challenge

In [ ]:
data = {
    "flag": "flag{ssh-example}"
}

r = s.post(deployer_host + f"/challenges/{challenge_id}/verify", timeout=20, json=data)
print("Test flag:", r.status_code, r.content)
r.raise_for_status()

### Stop the challenge using the API

In [ ]:
r = s.post(deployer_host + "/challenges/" + challenge_id + "/stop", timeout=20)
print("stop challenge:", r.status_code, r.content)
r.raise_for_status()

### Stop the test using the API

In [ ]:
r = s.post(deployer_host + "/solutions/" + challenge_id + "/stop", timeout=20)
print("stop test:", r.status_code, r.content)
r.raise_for_status()

### Get Logs

In [ ]:
r = s.get(deployer_host + "/challenges/" + challenge_id + "/logs", timeout=20)
print("logs:", r.status_code, r.content)
r.raise_for_status()

### SSH command
```bash
ssh test@ssh.<subdomain> -p 8022 -J bastion@ctf.jacopomauro.com:443 -L 8080:web1.ctf:80
```